In [10]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd

#creating a class to scrape data from Rotten Tomatoes
class RottenTomatoesScraper:
    def __init__(self,url):
        #reinstalling updated chromedriver just in case 
        import chromedriver_autoinstaller as chromedriver
        chromedriver.install()
        
        self.url = url
        self.driver = webdriver.Chrome()
        self.driver.maximize_window()
        self.driver.get(self.url)
        
        self.cookie_selector = '//*[@id="onetrust-accept-btn-handler"]'
        self.next_button_selector = '//*[@id="main-page-content"]/div/div[5]/button'
        self.main_page_movie_selector = 'js-tile-link'
        self.click_movie_selector = '//*[@id="main-page-content"]/div/div[3]/div/div/a[{}]/tile-dynamic/div'
        self.movie_info_selector = '//*[@id="topSection"]/div[1]/score-board/p'
        self.movie_description_selector = '//*[@id="movieSynopsis"]'
        self.extra_movie_info_selector = '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul'
        
        try:
            cookie_accept = self.driver.find_element(by = By.XPATH, value = self.cookie_selector)
            cookie_accept.click()
            time.sleep(1)
        except:
            time.sleep(2)
            cookie_accept = self.driver.find_element(by = By.XPATH, value = self.cookie_selector)
            cookie_accept.click()
            time.sleep(1)
            
    def load_pages(self, num_pages = 1):
        self.num_pages = num_pages
        for pages in range(num_pages - 1): 
            next_button = self.driver.find_element(by = By.XPATH, value = self.next_button_selector)
            next_button.click()
            time.sleep(4)
        
        self.movies = self.driver.find_elements(by = By.CLASS_NAME, value = self.main_page_movie_selector)
        self.movies_text = [movie.text for movie in self.movies]
        self.driver.execute_script("document.body.scrollIntoView(true);")
        
    def scrape_movies(self):
        self.n_movies = 30*self.num_pages
        movie_information = []
        for movie in range(1,self.n_movies - 1):
            movie_dict = {}
            movie_link =self.driver.find_element(by = By.XPATH, value = self.click_movie_selector.format(movie) )
            print(self.click_movie_selector.format(movie))
            movie_link.click()
            time.sleep(4)
            try:
                movie_info = self.driver.find_element(by = By.XPATH, value = self.movie_info_selector )
                movie_dict['movie_info'] = movie_info.text 
            except:
                movie_dict['movie_info'] = 'Not available'

            if "WATCH THE TRAILER" in self.movies_text[movie - 1]:
                movie_dict["movie_name"] = self.movies_text[movie - 1].split('\n')[3]
                movie_dict["critics_score"] = self.movies_text[movie - 1].split('\n')[1]
                movie_dict["audience_score"] = self.movies_text[movie - 1].split('\n')[2]
            else:
                movie_dict["movie_name"] = self.movies_text[movie - 1].split('\n')[2]
                movie_dict["critics_score"] = self.movies_text[movie - 1].split('\n')[0]
                movie_dict["audience_score"] = self.movies_text[movie - 1].split('\n')[1]
            

            movie_dict['Description'] = self.driver.find_element(by = By.XPATH, value = self.movie_description_selector).text
            extra_movie_info  = self.driver.find_element(by = By.CSS_SELECTOR, value = self.extra_movie_info_selector).text
            split_movie_info = extra_movie_info .split('\n')
            for i in range(len(split_movie_info)):
                temp_split = split_movie_info[i].split(': ')
                movie_dict[temp_split[0]] = temp_split[1]
                
            

            movie_information.append(movie_dict)
            self.driver.back()
            time.sleep(2)
            
        return pd.DataFrame.from_dict(movie_information)



    

In [11]:
scraper_object = RottenTomatoesScraper("https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=1")
scraper_object.load_pages()
movie_dataframe = scraper_object.scrape_movies()
display(movie_dataframe)

//*[@id="main-page-content"]/div/div[3]/div/div/a[1]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[2]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[3]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[4]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[5]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[6]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[7]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[8]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[9]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[10]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[11]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[12]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[13]/tile-dynamic/div
//*[@id="main-page-content"]/div/div[3]/div/div/a[14]/tile-dynamic/div
//*[@id="main-p

,movie_info,movie_name,critics_score,audience_score,Description,Genre,Original Language,Director,Release Date (Streaming),Runtime,Rating,Producer,Writer,Release Date (Theaters),Box Office (Gross USA),Distributor,Sound Mix,Aspect Ratio,View the collection
0,"2022, Mystery & thriller/Horror, 1h 20m",The Ghosts of Monday,--,--,Mark Huberman (Vikings Valhalla) stars as a te...,"Mystery & Thriller, Horror",English,Francesco,"Jan 23, 2023",1h 20m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2022, Horror/Mystery & thriller, 1h 42m",M3GAN,94%,79%,"M3GAN is a marvel of artificial intelligence, ...","Horror, Mystery & Thriller, Sci-Fi, Comedy",English,Gerard Johnstone,"Jan 24, 2023",1h 42m,PG-13 (Terror|Some Strong Language|A Suggestiv...,"Jason Blum, James Wan, Couper Samuelson",Akela Cooper,"Jan 6, 2023 Wide",$73.2M,Universal Pictures,Dolby Digital,Scope (2.35:1),NaN
2,"2022, Horror/Mystery & thriller, 1h 47m",The Menu,88%,76%,A couple (Anya Taylor-Joy and Nicholas Hoult) ...,"Horror, Mystery & Thriller, Comedy",English,Mark Mylod,"Jan 3, 2023",1h 47m,R (Some Sexual References|Language Throughout|...,"Adam McKay, Betsy Koch, Will Ferrell","Seth Reiss, Will Tracy","Nov 18, 2022 Wide",$38.4M,Searchlight Pictures,"Dolby Digital, Dolby Atmos",Scope (2.35:1),NaN
3,"2023, Horror/Mystery & thriller, 1h 32m",There's Something Wrong with the Children,63%,30%,When Margaret (Alisha Wainwright) and Ben (Zac...,"Horror, Mystery & Thriller",English,Roxanne Benjamin,"Jan 17, 2023",1h 32m,NaN,"Paige Pemberton, Paul Uddo","T.J. Cimfel, David White",NaN,NaN,NaN,NaN,NaN,NaN
4,"2022, Comedy/Mystery & thriller, 1h 47m",Vengeance,81%,86%,"Vengeance, the directorial debut from writer a...","Comedy, Mystery & Thriller, Drama, Horror",English,B.J. Novak,"Sep 16, 2022",1h 47m,R (Language and Brief Violence),"Jason Blum, Adam Hendricks, Greg Gilreath",B.J. Novak,"Jul 29, 2022 Wide",$4.2M,Focus Features,NaN,NaN,NaN
5,"2022, Horror/Mystery & thriller, 1h 23m",Sick,87%,66%,As the pandemic steadily brings the world to a...,"Horror, Mystery & Thriller",English,John Hyams,"Jan 13, 2023",1h 23m,R (Terror|Language Throughout|Strong Violence|...,"Ben Fast, Kevin Williamson, Bill Block","Kevin Williamson, Katelyn Crabb",NaN,NaN,NaN,NaN,NaN,NaN
6,"2022, Comedy/Drama, 2h 16m",White Noise,63%,32%,"At once hilarious and horrifying, lyrical and ...","Comedy, Drama, Horror",English,Noah Baumbach,"Dec 30, 2022",2h 16m,R (Brief Violence and Language),"Noah Baumbach, David Heyman, Uri Singer",Noah Baumbach,"Nov 25, 2022 Limited",NaN,Netflix,Dolby Digital,Scope (2.35:1),NaN
7,"2022, Horror/Mystery & thriller, 1h 33m",The Offering,90%,79%,In the wake of a young Jewish girl's disappear...,"Horror, Mystery & Thriller",English,Oliver Park,"Jan 13, 2023",1h 33m,R (Violence),"Les Weldon, Yariv Lerner, Jeffrey Greenstein, ...",Hank Hoffman,"Jan 13, 2023 Limited",NaN,Decal,NaN,NaN,NaN
8,"2022, Horror/Mystery & thriller, 1h 45m",X,94%,75%,"In 1979, a group of young filmmakers set out t...","Horror, Mystery & Thriller",English,Ti West,"Apr 14, 2022",1h 45m,R (Gore|Drug Use|Strong Sexual Content|Graphic...,"Jacob Jaffke, Kevin Turen, Harrison Kreiss",Ti West,"Mar 18, 2022 Wide",$11.6M,A24,NaN,NaN,NaN
9,"2022, Horror/Mystery & thriller, 1h 42m",Barbarian,92%,71%,"Traveling to Detroit for a job interview, a yo...","Horror, Mystery & Thriller",English,Zach Cregger,"Oct 25, 2022",1h 42m,R (Language Throughout|Disturbing Material|Nud...,"Arnon Milchan, Roy Lee, Raphael Margules, J.D....",Zach Cregger,"Sep 9, 2022 Wide",$40.8M,20th Century Studios,NaN,Flat (1.85:1),NaN


In [12]:
movie_dataframe.to_csv("horror_movie_data.csv")